In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-13"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
14116,2024-03-13,Eua Nba,20:00,Detroit Pistons,Toronto Raptors,1.74,1.93,232.5,1.83,1.97,-3.5,2.03,2.45,fiXeIGSi,0.574713,0.518135,0.546448,0.507614,0.092847,0.4,1.0,NaN,NaN,501.052,164.489925,0.328289,240.828,34.489829,0.143214,418.066,335.076,259.60,202.96,0.0,0.0,0.0,0.0,0.073215,0.052103,0.132583,3.99,0.798,0.927318,0.302425,0.3,-0.002425,-3.60,-0.720,-1.291667,0.379682,0.2,-0.179682
14117,2024-03-13,Eua Nba,20:30,Miami Heat,Denver Nuggets,2.39,1.62,214.5,1.90,1.96,2.5,1.96,1.38,pEmne1YO,0.418410,0.617284,0.526316,0.510204,0.035694,0.2,0.2,NaN,NaN,334.718,102.811290,0.307158,197.170,41.409927,0.210021,217.550,201.136,311.64,203.49,0.0,0.0,0.0,0.0,0.271557,0.021983,0.245582,-0.21,-0.042,-33.095238,0.585731,0.4,-0.185731,-1.68,-0.336,-1.845238,0.560290,0.5,-0.060290
14118,2024-03-13,Eua Nba,23:00,Sacramento Kings,Los Angeles Lakers,2.04,1.83,238.5,1.82,1.98,1.5,1.94,1.68,2142sU0R,0.490196,0.546448,0.549451,0.505051,0.036644,0.8,0.4,NaN,NaN,194.000,61.621508,0.317637,257.084,34.526996,0.134302,184.860,261.400,229.62,273.76,0.0,0.0,0.0,0.0,0.076740,0.059546,0.101573,0.94,0.188,5.531915,0.720541,0.6,-0.120541,1.16,0.232,3.577586,0.433988,0.4,-0.033988
14119,2024-03-13,Austrália Nbl,05:30,Melbourne United,Illawarra Hawks,1.31,3.51,182.5,1.87,1.79,-8.5,1.93,7.00,8GvZKIgL,0.763359,0.284900,0.534759,0.558659,0.048259,0.6,0.4,NaN,NaN,138.064,23.887701,0.173019,216.432,68.239431,0.315293,132.566,257.106,119.60,315.81,0.0,0.0,0.0,0.0,0.645492,0.030912,0.802919,0.16,0.032,9.687500,0.000000,0.0,0.000000,0.65,0.130,19.307692,0.358979,0.5,0.141021
14120,2024-03-13,Austrália Nbl,07:30,Perth Wildcats,Tasmania JackJumpers,1.98,1.83,178.5,1.87,1.79,8.5,1.27,1.26,S2zODDmR,0.505051,0.546448,0.534759,0.558659,0.051499,0.4,0.2,NaN,NaN,168.556,41.037985,0.243468,137.062,24.181622,0.176428,140.226,147.814,162.87,160.38,0.0,0.0,0.0,0.0,0.055678,0.030912,0.005590,-0.47,-0.094,-10.425532,0.671798,0.8,0.128202,-2.29,-0.458,-1.812227,0.595458,0.6,0.004542
14121,2024-03-13,Coreia Do Sul Kbl,07:00,Anyang JungKwanJang,Goyang Sono,1.85,1.92,169.5,1.88,1.86,-1.5,1.94,2.08,rc5LAkUr,0.540541,0.520833,0.531915,0.537634,0.061374,0.6,0.8,NaN,NaN,191.500,96.492876,0.503879,180.728,50.376561,0.278742,217.688,317.256,180.32,173.72,0.0,0.0,0.0,0.0,0.026259,0.007563,0.049251,1.31,0.262,3.244275,0.378977,0.4,0.021023,2.50,0.500,1.840000,0.337689,0.2,-0.137689
14122,2024-03-13,Espanha Leb Ouro,16:30,Lleida,San Pablo Burgos,1.94,1.78,163.5,1.80,1.86,1.5,1.80,1.64,jFpFy7vc,0.515464,0.561798,0.555556,0.537634,0.077262,1.0,0.6,NaN,NaN,152.050,24.686337,0.162357,105.338,22.381622,0.212474,124.632,113.642,166.47,90.85,0.0,0.0,0.0,0.0,0.060826,0.023184,0.065777,0.25,0.050,18.800000,0.000000,0.0,0.000000,-1.51,-0.302,-2.582781,0.000000,0.0,0.000000
14123,2024-03-13,Eua Ncaa,12:30,George Mason,Saint Josephs,1.91,1.91,136.5,1.80,1.95,-1.5,1.95,2.05,x2i6niZd,0.523560,0.523560,0.555556,0.512821,0.047120,0.6,0.6,NaN,NaN,128.620,28.914347,0.224804,137.832,19.272900,0.139829,103.266,146.274,108.90,124.88,0.0,0.0,0.0,0.0,0.000000,0.056569,0.035355,0.42,0.084,10.833333,0.789569,0.8,0.010431,-1.35,-0.270,-3.370370,0.502778,0.4,-0.102778
14124,2024-03-13,Eua Ncaa,13:00,Virginia Tech,Florida State,1.63,2.38,152.5,1.91,1.91,-4.5,2.00,3.30,GhjAoBl3,0.613497,0.420168,0.523560,0.523560,0.033665,0.4,1.0,NaN,NaN,188.130,149.132606,0.792710,166.936,34.292813,0.205425,128.060,186.968,128.25,209.25,0.0,0.0,0.0,0.0,0.264504,0.0

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:00,Eua Nba,Detroit Pistons,Toronto Raptors,232.5,1.83,1.0000,Over
1,05:30,Austrália Nbl,Melbourne United,Illawarra Hawks,182.5,1.87,1.0000,Over
2,07:00,Coreia Do Sul Kbl,Anyang JungKwanJang,Goyang Sono,169.5,1.88,1.0000,Over
3,12:30,Eua Ncaa,George Mason,Saint Josephs,136.5,1.80,0.9943,Over
4,13:00,Eua Ncaa,Virginia Tech,Florida State,152.5,1.91,1.0000,Over
5,14:00,Eua Ncaa,Wichita State,Rice,139.5,1.80,1.0000,Over
6,16:00,Eua Ncaa,Alcorn State,Alabama A&M,140.5,1.91,1.0000,Over
7,16:00,Eua Ncaa,Temple,UTSA Roadrunners,150.5,1.80,0.9938,Over
8,17:00,Eua Ncaa,Butler,Xavier,150.5,1.91,1.0000,Over
9,19:30,Eua Ncaa,Maryland,Rutgers,125.5,1.91,1.0000,Over
